In [2]:
from utils import get_config, join_sets, load_model, print_log, save_model, load_model_dict
from data import get_model_input
from model import get_best_model
from superdebug import debug
config = get_config("configs/debug_med_ID_sim_new_votes.yml", "_manipulation")
config["batch_size"] = 1
all_feature_columns, target, train_model_input, test_model_input, feature_names, original_feature_map, max_voted_users, train_data, test_data, test_data_info = get_model_input(config)
model, user_embedding = get_best_model(config, all_feature_columns, feature_names)
model = model.to(model.device)

GPU ready...
Smart using cuda:0
------------------ 2022-07-25 23:25:48 ------------------
DEBUG: at /home/TableSense/largedisk/wanrong/Curation-Modeling/data.py:283 get_model_input
Loading prepared data...
------------------ 2022-07-25 23:25:48 ------------------


In [4]:
import numpy as np
from train import get_data_loader, pad_sequences
model = model.eval()
if "UPVOTED_USERS" in test_model_input:
    test_model_input["UPVOTED_USERS"] = pad_sequences(test_model_input["UPVOTED_USERS"], maxlen=max_voted_users, padding='post')
    test_model_input["DOWNVOTED_USERS"] = pad_sequences(test_model_input["DOWNVOTED_USERS"], maxlen=max_voted_users, padding='post')
test_model_input_list = [test_model_input[feature] for feature in model.feature_index if feature in test_model_input]

for i in range(len(test_model_input_list)):
    if len(test_model_input_list[i].shape) == 1:
        test_model_input_list[i] = np.expand_dims(test_model_input_list[i], axis=1)

tensor_data, test_loader = get_data_loader(model.lm_encoder is not None, test_model_input_list, test_model_input.get("input_ids", None),test_model_input.get("token_type_ids", None) ,test_model_input.get("attention_mask", None), test_data[target].values, None, shuffle=False, batch_size=config["batch_size"])
test_input_list = []
for test_input in test_loader:
    test_input_list.append(test_input)

In [ ]:
from matplotlib import interactive
import torch
from tqdm import tqdm
from train import convert_CTR_model_input, to_device, modify_updown_voted_users
y = test_data[target].values
print("Total user num:", user_embedding.shape[0])
for test_input in tqdm(test_input_list):
    CTR_model_input = convert_CTR_model_input(model, test_input, sample_voted_users=True, interactive = True)
    while CTR_model_input is not None:
        x, _y, weight = CTR_model_input
        print("User:", int(x[:,model.feature_index["USERNAME"][0]:model.feature_index["USERNAME"][1]]))
        y_pred = model(x).cpu().data.numpy()  # .squeeze()
        print(y_pred[0][0], "Upvote" if y_pred[0][0] > 0.5 else "Downvote")
        print("---------------------")
        CTR_model_input = convert_CTR_model_input(model, test_input, sample_voted_users=True, interactive = True)